# NX-421 Mini-Project — Part 1 (Steps 3–6)

This notebook continues after preprocessing and follows the same approach as the labs and teammates' notebooks.
It assumes you already have a preprocessed 4D fMRI time series for subject 101410 (concatenated runs, motion-corrected, smoothed) and event files.

We will:
- Build and visualize the experimental design matrix (3).
- Fit a first-level GLM and report statistical maps per task regressor (4).
- Define and apply a hand vs. foot contrast, and save the contrast map (5).
- Overlay the contrast map with the AAL atlas and report regions with maximal contrast (6).

Notes:
- Keep parameters and functions in line with nx421-course/Week05 GLM lab (nilearn FirstLevelModel, spm HRF, ar1 noise).
- This notebook does not re-run preprocessing. Point `FUNC_IMG` to your final preprocessed 4D image.
- Adjust paths in the Parameters cell below to match your VM and data layout.


## Parameters
Set input/output paths here.
- `FUNC_IMG`: final preprocessed 4D BOLD timeseries used for GLM.
- `EVENT_FILES`: list of per-run event CSVs, each with `onset`, `duration`, `trial_type`.
  If you already have a single concatenated events CSV, pass it as a single-item list.
- If you concatenated runs into one functional image, you must offset onsets of later runs before GLM.
  Provide either `RUN_IMG_FILES` (original per-run 4D files) or `RUN_SCANS` (n_volumes per run) so we can compute offsets based on TR.
- If motion confounds are available (e.g., MCFLIRT `.par` to regress), point `CONFOUNDS_TSV` to a TSV or CSV you prepared; otherwise leave `None`.


In [ ]:
import os, os.path as op

# Adjust these paths for your VM
DATA_ROOT = op.abspath('NSSP-Mini-Project-1/data')  # change if needed
OUTPUT_DIR = op.abspath('NSSP-Mini-Project-1/results_part1_glm')
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Final preprocessed 4D image (concatenated, motion-corrected, smoothed).
# Example placeholder; update to your produced filename:
FUNC_IMG = op.join(DATA_ROOT, 'sub-101410_task-motor_preproc_concat_bold.nii.gz')

# Event files per run. If you already have a single concatenated events file,
# pass it as a single-item list and leave RUN_IMG_FILES/RUN_SCANS empty.
EVENT_FILES = [
    op.join(DATA_ROOT, 'sub-101410_task-motor_run-LR_events.csv'),
    op.join(DATA_ROOT, 'sub-101410_task-motor_run-RL_events.csv'),
]

# Optional: original per-run 4D images to infer run lengths for onset offsets.
# If provided, we use these to compute offsets. Otherwise, use RUN_SCANS or assume events are already concatenated.
RUN_IMG_FILES = [
    op.join(DATA_ROOT, 'tfMRI_MOTOR_LR.nii.gz'),
    op.join(DATA_ROOT, 'tfMRI_MOTOR_RL.nii.gz'),
]

# Alternative: provide number of scans per run (if RUN_IMG_FILES are not available).
RUN_SCANS = []  # e.g., [284, 284]

# Optional confounds file (e.g., motion regressors). Leave as None if not used.
CONFOUNDS_TSV = None  # op.join(DATA_ROOT, 'sub-101410_task-motor_confounds.tsv')


In [ ]:
# Imports consistent with Week05 GLM lab
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
from nilearn.glm.first_level import FirstLevelModel, make_first_level_design_matrix
from nilearn import plotting, image
from nilearn.datasets import fetch_atlas_aal, load_mni152_template

# Masker import for region-wise extraction (new vs old nilearn)
try:
    from nilearn.maskers import NiftiLabelsMasker
except Exception:
    from nilearn.input_data import NiftiLabelsMasker

import warnings
warnings.filterwarnings('ignore')


## Load functional image and TR
We read the preprocessed concatenated 4D BOLD and extract TR from header.


In [ ]:
assert op.isfile(FUNC_IMG), f'Functional image not found: {FUNC_IMG}'
fmri_img = nib.load(FUNC_IMG)
n_scans = fmri_img.shape[-1]
zooms = fmri_img.header.get_zooms()
TR = float(zooms[3]) if len(zooms) > 3 else 0.72  # default to 0.72s if missing
print(f'Loaded functional: {FUNC_IMG}')
print(f'Shape: {fmri_img.shape}, TR: {TR:.3f} s, n_scans: {n_scans}')


## Load events and adjust onsets for concatenation
- Expects columns: `onset`, `duration`, `trial_type`.
- If multiple runs provided, onsets for subsequent runs are offset by the preceding run durations (`n_scans_run * TR`).
- If neither `RUN_IMG_FILES` nor `RUN_SCANS` is given, events are assumed already concatenated.


In [ ]:
def _load_events(paths):
    evs = []
    for idx, p in enumerate(paths):
        assert op.isfile(p), f'Events file not found: {p}'
        df = pd.read_csv(p)
        required = {'onset','duration','trial_type'}
        missing = required - set(df.columns)
        if missing:
            raise ValueError(f'Missing columns in {p}: {missing}')
        df = df[['onset','duration','trial_type']].copy()
        df['run_idx'] = idx
        evs.append(df)
    return evs

def _get_run_scans(run_img_files, fallback_scans):
    scans = []
    if run_img_files and all(op.isfile(r) for r in run_img_files):
        for r in run_img_files:
            img = nib.load(r)
            scans.append(img.shape[-1])
    elif fallback_scans:
        scans = list(fallback_scans)
    return scans

def adjust_events_for_concatenation(events_list, run_scans, tr):
    # If single events file, nothing to adjust
    if len(events_list) == 1:
        return events_list[0].drop(columns=['run_idx'], errors='ignore').reset_index(drop=True)
    # If multiple, compute cumulative onset offsets
    if not run_scans or len(run_scans) != len(events_list):
        # Assume already concatenated
        print('No run lengths provided; assuming events are already concatenated.')
        return pd.concat(events_list, ignore_index=True).drop(columns=['run_idx'], errors='ignore').reset_index(drop=True)
    offsets = np.cumsum([0] + [int(s)*tr for s in run_scans[:-1]]).astype(float)
    adjusted = []
    for df in events_list:
        ridx = int(df['run_idx'].iloc[0])
        off = float(offsets[ridx])
        tmp = df.copy()
        tmp['onset'] = tmp['onset'] + off
        adjusted.append(tmp)
    out = pd.concat(adjusted, ignore_index=True)
    return out.drop(columns=['run_idx']).reset_index(drop=True)

raw_events = _load_events(EVENT_FILES)
run_scans = _get_run_scans(RUN_IMG_FILES, RUN_SCANS)
events = adjust_events_for_concatenation(raw_events, run_scans, TR)
print('Unique trial types:', sorted(events['trial_type'].unique()))
events.head()


## 3. Design matrix
Following the Week05 lab, we construct the design matrix using SPM HRF, AR(1) noise, and cosine drift (high-pass=0.01 Hz).
We display the design matrix for reporting.


In [ ]:
from nilearn.plotting import plot_design_matrix

frame_times = np.arange(n_scans) * TR
design_matrix = make_first_level_design_matrix(
    frame_times,
    events=events,
    hrf_model='spm',
    drift_model='cosine',
    high_pass=0.01
)

# Optionally add confounds (e.g., motion params, outlier regressors) as extra columns
if CONFOUNDS_TSV is not None and op.isfile(CONFOUNDS_TSV):
    conf = pd.read_csv(CONFOUNDS_TSV, sep='	' if CONFOUNDS_TSV.endswith('.tsv') else ',')
    # Keep numeric columns only and align rows to n_scans
    conf = conf.select_dtypes(include=[np.number])
    if conf.shape[0] != n_scans:
        print(f'Warning: confounds rows ({conf.shape[0]}) != n_scans ({n_scans}); attempting to trim/pad.')
        conf = conf.iloc[:n_scans, :]
        if conf.shape[0] < n_scans:
            # pad with zeros
            pad_rows = n_scans - conf.shape[0]
            conf = pd.concat([conf, pd.DataFrame(np.zeros((pad_rows, conf.shape[1])), columns=conf.columns)], ignore_index=True)
    # Concatenate along columns
    design_matrix = pd.concat([design_matrix.reset_index(drop=True), conf.reset_index(drop=True)], axis=1)
    print(f'Added confounds to design matrix: {list(conf.columns)[:6]}...')
ax = plot_design_matrix(design_matrix)
ax.set_title('Design matrix')
plt.tight_layout()
plt.savefig(op.join(OUTPUT_DIR, 'design_matrix.png'), dpi=150)
plt.show()
design_matrix.head()


## 4. First-level GLM and statistical maps
We fit a `FirstLevelModel` with the above design matrix and produce z-maps for each task regressor (trial_type).


In [ ]:
fmri_glm = FirstLevelModel(
    t_r=TR,
    noise_model='ar1',
    standardize=False,
    hrf_model='spm',
    drift_model='cosine',
    high_pass=0.01
)
fmri_glm = fmri_glm.fit(fmri_img, design_matrices=[design_matrix])

# Identify task regressors (exclude drift/cosine columns)
task_cols = [c for c in design_matrix.columns if c not in ('constant',) and not c.startswith('drift')]
print('Task regressors:', task_cols)

zmap_paths = {}
for col in task_cols:
    zmap = fmri_glm.compute_contrast(col, output_type='z_score')
    out_nii = op.join(OUTPUT_DIR, f'zmap_{col}.nii.gz')
    zmap.to_filename(out_nii)
    zmap_paths[col] = out_nii
    display = plotting.plot_stat_map(
        zmap, bg_img=load_mni152_template(), threshold=3.1, display_mode='z', cut_coords=7,
        title=f'Z-map: {col}'
    )
    display.savefig(op.join(OUTPUT_DIR, f'zmap_{col}.png'))
    display.close()
zmap_paths


## 5. Hand vs. Foot contrast
We define a contrast averaging left/right hands vs. left/right feet, in line with the lab’s design.
If your `trial_type` naming differs, adjust the `hand_patterns` and `foot_patterns` below.


In [ ]:
# Auto-detect columns for hands/feet via substring match
hand_patterns = ['hand']
foot_patterns = ['foot']

hand_cols = [c for c in task_cols if any(p.lower() in c.lower() for p in hand_patterns)]
foot_cols = [c for c in task_cols if any(p.lower() in c.lower() for p in foot_patterns)]
print('Detected hand columns:', hand_cols)
print('Detected foot columns:', foot_cols)
assert hand_cols and foot_cols, 'Could not detect hand/foot columns. Please adjust patterns.'

# Build contrast as dict over design columns
contrast_def = {c: 0.0 for c in design_matrix.columns}
for c in hand_cols:
    contrast_def[c] += 1.0 / len(hand_cols)
for c in foot_cols:
    contrast_def[c] -= 1.0 / len(foot_cols)

# Remove drift entries (remain 0)
hand_vs_foot_z = fmri_glm.compute_contrast(contrast_def, output_type='z_score')
contrast_nii = op.join(OUTPUT_DIR, 'zmap_hand_vs_foot.nii.gz')
hand_vs_foot_z.to_filename(contrast_nii)
display = plotting.plot_stat_map(
    hand_vs_foot_z, bg_img=load_mni152_template(), threshold=3.1, display_mode='z', cut_coords=7,
    title='Hand > Foot (Z)'
)
display.savefig(op.join(OUTPUT_DIR, 'zmap_hand_vs_foot.png'))
display.close()
print('Contrast saved to:', contrast_nii)
contrast_def


## 6. Overlay with AAL atlas and report regions
We overlay the contrast map with AAL parcellation and compute mean z-value per region, reporting regions with maximal contrast.


In [ ]:
# Fetch AAL atlas and resample to functional image space
aal = fetch_atlas_aal()
atlas_img = nib.load(aal['maps']) if isinstance(aal['maps'], str) else aal['maps']
atlas_res = image.resample_to_img(atlas_img, fmri_img, interpolation='nearest')
labels = list(aal['labels'])

# Compute region-wise mean z from the contrast map
masker = NiftiLabelsMasker(labels_img=atlas_res, standardize=False)
reg_vals = masker.fit_transform(hand_vs_foot_z)  # shape (1, n_regions)
reg_vals = reg_vals.ravel()

region_df = pd.DataFrame({'label': labels, 'z': reg_vals})
region_df_sorted = region_df.sort_values('z', ascending=False)
region_df_sorted.to_csv(op.join(OUTPUT_DIR, 'regions_hand_vs_foot_AAL.csv'), index=False)

print('Top positive regions (Hand > Foot):')
display(region_df_sorted.head(10))
print('Top negative regions (Foot > Hand):')
display(region_df_sorted.tail(10))

# Visualization: add AAL as translucent overlay on contrast map
disp = plotting.plot_stat_map(
    hand_vs_foot_z, bg_img=load_mni152_template(), threshold=3.1, display_mode='z', cut_coords=7,
    title='Hand > Foot (Z) with AAL overlay'
)
disp.add_overlay(atlas_res, cmap='tab20', alpha=0.15)
disp.savefig(op.join(OUTPUT_DIR, 'zmap_hand_vs_foot_with_AAL.png'))
disp.close()


## Reporting Notes (for your write-up)
- Design matrix: include the saved `design_matrix.png`. Describe conditions and drift.
- GLM: report z-maps for each task regressor saved as `zmap_<regressor>.png`.
- Contrast: report the vector weights (printed above) and show `zmap_hand_vs_foot.png`.
- AAL overlay: show `zmap_hand_vs_foot_with_AAL.png` and list top regions from `regions_hand_vs_foot_AAL.csv`.
- Methods match Week05 lab: nilearn FirstLevelModel with `hrf_model='spm'`, `noise_model='ar1'`, `drift_model='cosine'`, `high_pass=0.01`.

If you included motion regressors or censored volumes, add them as additional columns in the design matrix (as in the lab) before fitting.
